In [ ]:
from Painter import Painter

# Keypoints

In [ ]:
name = 'sacre_coeur_A.jpg'

In [ ]:
Painter.show_keypoints(name)

In [ ]:
Painter.show_patches(name)

# Matches

In [ ]:
name_a = 'sacre_coeur_A.jpg'
name_b = 'sacre_coeur_B.jpg'

In [ ]:
Painter.show_patches(name_a, name_b, num_points=30)

In [ ]:
Painter.show_filtered_keypoint_matches(name_a, name_b, num_points=30)